### Data prep

#### Source data

Source data for the 10B benchmark contains 3 types:

 - Customer data
   - name, address, customer number ...
   - Identity card (jpeg or pdf)
 - Account data
   - Account number (+customer data)
   - Letter for opening this account
 - Statements
   - Month/Year
   - Operations
   - PDF 
 
 For Customers and Account data:
  - meta-data has been pre-generated
     - https://10b-benchmark-data.s3.amazonaws.com/all-id-cards.csv
     - https://10b-benchmark-data.s3.amazonaws.com/all-accounts.csv
  - binaries have been pre-generated
     - stored on a snowball then on a S3 bucket
 
 For Statements, data is generated 100% dynamically:
 
  - the importer generate the statement data on the fly
  - the custom BlobManager return the corresponding PDF
  
 #### Volume and cardinality
 
 ##### Customers
 
 all-id-cards.csv contains 89,997,827 clients
 
 When imported, this will generate 89,997,827*2=179,995,654 documents 
 
  - 89,997,827 `Customer` documents
  - 89,997,827 `IDcard` documents
 
 ##### Accounts
 
 all-accounts.csv contains 179,998,862 accounts
 
 In average this makes 2 accounts per customers, but some customer will have 1 and some will have 3.
 
 When imported, this will generate 179,998,862*2=359,997,724 documents 
 
  - 179,998,862 `Account` documents
  - 179,998,862 `Correspondance<State>` documents
 
 ##### Statements
 
 The statements are generated using a parameter that correspond to the number of months.
 
 Generating 179,998,862 statements for 1 months, will generate 1 statement for each account.
 Generating 179,998,862*2 statements for 2 months, will generate 2 statements for each account.
 Generating 179,998,862*60 statements for 60 months, will generate 60 statements for each account.
 
 ##### Total
 
  - 89,997,827 `Customer` documents
  - 89,997,827 `IDcard` documents
  - 179,998,862 `Account` documents
  - 179,998,862 `Correspondance<State>` documents
  - 179,998,862*60 `Statement` documents
  
 11,339,925,098 documents
 

#### Extracting a smaller dataset

It is possible to extract a smaller dataset, here is an example:

Let's say we want to start with 100,000 customers:

In [13]:
!head -100000 ../data/sources/all-id-cards.csv > ../data/100m-customers.csv
!wc -l ../data/100m-customers.csv 

100000 ../data/100m-customers.csv


Now we need about 2 times more accounts, but because it is randomly distributed, we will take 3 times more lines from accounts than from customer and will filter as a second step. 

In [22]:
!head -300000 ../data/sources/all-accounts.csv > ../data/300m-accounts.csv
!wc -l ../data/300m-accounts.csv 

300000 ../data/300m-accounts.csv


In [23]:
!csvcheck.sh -i ../data/100m-customers.csv -a ../data/300m-accounts.csv 

Reading the ids
Procesing /nxbench/notebooks/../data/100m-customers.csv
collected 100000 keys
Procesing /nxbench/notebooks/../data/300m-accounts.csv
 line: 300000 missing:99916
line processed: 300000 # accounts missing:99916


csvcheck will match the accounts with the idcards and split them in:
 - verified-accounts.csv : the accounts that correspond to an id
 - accounts-withoutIDCard.csv : the orphaned accounts

In [24]:
!ls *.csv
!wc -l verified-accounts.csv

accounts-withoutIDCard.csv  verified-accounts.csv
200084 verified-accounts.csv


In [25]:
%%bash
rm accounts-withoutIDCard.csv
rm ../data/300m-accounts.csv
mv verified-accounts.csv ../data/200084-accounts.csv
ls ../data/ 

0002812b91635a426fb6ebb0515c0244
100m-customers.csv
200084-accounts.csv
200m-accounts.csv
faces
id-cards-meta-data
id-cards.zip
letters
letters-import-clean.zip
letters-import.zip
sources
